Turns out, CLASS has restrictions on how many components the model needs to run. Theoretically we can just change certain densities to really small number, but there are some restrictions to that as well.\
The textbook is very mathametical, which allows us to obstain results which are unphysical. However, CLASS uses empirical data as initial conditions, which implies the existance of unviolatable physical laws

### Approach the various limits CLASS has on the different parameters
By plotting the background evolution at these limits, we can simulate as close as we can the simplied model

In [2]:
import numpy as np
from classy import Class
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve

In [3]:
def config_default():
    config_default = {'h': 0.67810, 'A_s': 2.100549e-09, 
                            'omega_b': 0.0238280, 
                            'omega_cdm': 0.1201075, 
                            'T_cmb': 2.7255 ,'z_pk': 0, 
                            'output' : 'tCl,pCl,lCl,mPk',
                            'lensing': 'yes', 'P_k_max_1/Mpc': 1., 'l_max_scalars': 2500,}
    return config_default

In [4]:
def find_viable_parameters(parameter, increment_ratio: float, max_iterations=50):
    
    config = config_default()
    start = config[parameter]
    viable_params = [start]  # Start with the initial value as viable
    param_increment = start*(1 + increment_ratio)
    param_decrement = start*(1 - increment_ratio)

    for _ in range(max_iterations):
        print(_)
        try:
            config.update({parameter: param_increment})
            cosmo_modified = Class() 
            cosmo_modified.set(config)
            cosmo_modified.compute()
            param_increment = param_increment*(1 + increment_ratio)  # Increment the parameter for the next iteration
        except Exception as e:
            # If an error occurs, print it and break out of the loop
            print(f"Stopped due to error: {e}")
            break
        viable_params.append(param_increment)


    for _ in range(max_iterations):
        print(_)
        try:
            config.update({parameter: param_decrement})
            cosmo_modified = Class() 
            cosmo_modified.set(config)
            cosmo_modified.compute()
            param_decrement = param_decrement*(1 - increment_ratio)
        except Exception as e:
            print(f"\n Stopped decrementing due to error: {e}")
            break
        viable_params.append(param_decrement)
    
    return viable_params

In [5]:
# argument: (paramter_name, increment_ratio)
viable_omega_b = find_viable_parameters('omega_b', 0.1)
viable_T_cmb = find_viable_parameters('T_cmb', 0.05)
# cdm has not detected bounds

0
1
2
3
4
5
Stopped due to error: 

Error in Class: thermodynamics_init(L:342) :error in thermodynamics_helium_from_bbn(ppr,pba,pth);
=>thermodynamics_helium_from_bbn(L:656) :condition (omega_b > omegab[num_omegab-1]) is true; You have asked for an unrealistic high value omega_b = 4.221276e-02. The corresponding value of the primordial helium fraction cannot be found in the interpolation table. If you really want this value, you should fix YHe to a given value rather than to BBN
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14

 Stopped decrementing due to error: 

Error in Class: thermodynamics_init(L:342) :error in thermodynamics_helium_from_bbn(ppr,pba,pth);
=>thermodynamics_helium_from_bbn(L:651) :condition (omega_b < omegab[0]) is true; You have asked for an unrealistic small value omega_b = 4.905974e-03. The corresponding value of the primordial helium fraction cannot be found in the interpolation table. If you really want this value, you should fix YHe to a given value rather than to BBN
0
1


In [6]:
viable_omega_b.sort()
viable_omega_b.pop(0)
viable_omega_b.pop(-1)
# the above method will include the fail instances, here manually eliminate them

viable_T_cmb.sort()
viable_T_cmb.pop(0)
viable_T_cmb.pop(-1)

%store viable_omega_b
%store viable_T_cmb

Stored 'viable_omega_b' (list)
Stored 'viable_T_cmb' (list)


In [7]:
viable_omega_b

[0.005451082106168107,
 0.006056757895742341,
 0.006729730995269268,
 0.00747747888363252,
 0.0083083098707028,
 0.009231455411892,
 0.010257172679879998,
 0.011396858533199997,
 0.012663176147999998,
 0.014070195719999998,
 0.015633550799999996,
 0.017370611999999997,
 0.019300679999999997,
 0.023828,
 0.02883188,
 0.031715068000000006,
 0.03488657480000001,
 0.03837523228000001]